# Part One

In [836]:
import pandas as pd

In [837]:
file = open('./Input/Day9_Input.txt', 'r')

contents = file.read()
data = contents.split("\n")

In [838]:
# Main grid (list of lists) that we'll expand dynamically
grid = [['sht']] # s = Starting point, h = Head, t = tail

In [839]:
def new_head_destination(head, direction, steps):
    row_ix = head[0]
    col_ix = head[1]
    curr_row_len = len(grid)
    curr_col_len = len(grid[0])

    # Remove 'h' from current position
    grid[row_ix][col_ix] = grid[row_ix][col_ix].replace('h','')

    # Left
    if direction == 'L':
        dest_col_ix = col_ix - steps

        if dest_col_ix < 0:
            # Expand to the left
            amt_of_cols_needed = 0 - (dest_col_ix)
            step_ct = 0

            while step_ct < amt_of_cols_needed:
                for row in grid:
                    row.insert(0, '')

                step_ct = step_ct + 1

            # We had to expand, so we know the column index is the farthest left
            head = [row_ix, 0]
        else:
            # Destination index exists
            head = [row_ix, dest_col_ix]
    # Right
    elif direction == 'R':
        dest_col_ix = col_ix + steps

        if dest_col_ix >= curr_col_len:
            # Expand to the right
            amt_of_cols_needed = (dest_col_ix + 1) - curr_col_len
            step_ct = 0

            while step_ct < amt_of_cols_needed:
                for row in grid:
                    row.append('')

                step_ct = step_ct + 1

        head = [row_ix, dest_col_ix]
    # Up
    elif direction == 'U':
        dest_row_ix = row_ix - steps

        if dest_row_ix < 0:
            # Expand up
            amt_of_cols_needed = 0 - (dest_row_ix)
            step_ct = 0

            while step_ct < amt_of_cols_needed:
                grid.insert(0, [])

                col_ct = 0

                while col_ct < curr_col_len:
                    grid[0].append('')

                    col_ct = col_ct + 1

                step_ct = step_ct + 1

            head = [0, col_ix]
        else:
            head = [dest_row_ix, col_ix]
    # Down
    else:
        dest_row_ix = row_ix + steps

        if dest_row_ix >= curr_row_len:
            # Expand down
            amt_of_cols_needed = (dest_row_ix + 1) - curr_row_len
            step_ct = 0

            while step_ct < amt_of_cols_needed:
                grid.append([])

                col_ct = 0

                while col_ct < curr_col_len:
                    grid[-1].append('')

                    col_ct = col_ct + 1

                step_ct = step_ct + 1

        head = [dest_row_ix, col_ix]

    new_row_ix = head[0]
    new_col_ix = head[1]

    # Add 'h' to new position
    if 'h' not in grid[new_row_ix][new_col_ix]:
        grid[new_row_ix][new_col_ix] = grid[new_row_ix][new_col_ix] + 'h'

    return head

def get_tail_position():
    for r in grid:
        row_ix = grid.index(r)
        for x in r:
            col_ix = r.index(x)
            if 't' in x:
                tail = [row_ix, col_ix]

    return tail

def mark_trail(row_ix, col_ix):
    if '#' not in grid[row_ix][col_ix] and 's' not in grid[row_ix][col_ix]:
        grid[row_ix][col_ix] = grid[row_ix][col_ix] + '#'

def set_tail(row_ix, col_ix):
    if 't' not in grid[row_ix][col_ix]:
        grid[row_ix][col_ix] = grid[row_ix][col_ix] + 't'

def new_tail_destination(head, tail, direction, recurse = False):
    head_row_ix = head[0]
    head_col_ix = head[1]

    tail_row_ix = tail[0]
    tail_col_ix = tail[1]

    # Remove 't' from current position
    grid[tail_row_ix][tail_col_ix] = grid[tail_row_ix][tail_col_ix].replace('t','')

    mark_trail(tail_row_ix, tail_col_ix)

    # Left
    if direction == 'L':
        if head_row_ix == tail_row_ix:
            # 'h' and 't' are on the same row
            target_col_ix = head_col_ix + 1
            steps = tail_col_ix - target_col_ix
            step_ct = 0
            step_ix = tail_col_ix - 1

            while step_ct < steps and step_ix >= target_col_ix:
                mark_trail(tail_row_ix, step_ix)

                step_ix = step_ix - 1
                step_ct = step_ct + 1

            tail = [tail_row_ix, target_col_ix]
        elif head_col_ix < tail_col_ix - 1:
            # 'h' has travelled at least two rows over
            # 't' will jump diagonally into h's row
            mark_trail(tail_row_ix, tail_col_ix)

            target_row_ix = head_row_ix
            tail = [target_row_ix, tail_col_ix - 1]
            new_tail_row_ix = tail[0]
            new_tail_col_ix = tail[1]

            set_tail(new_tail_row_ix, new_tail_col_ix)

            if head_col_ix < new_tail_col_ix:
                tail = new_tail_destination(head, tail, direction, True)
    # Right
    elif direction == 'R':
        if head_row_ix == tail_row_ix:
            target_col_ix = head_col_ix - 1
            steps = tail_col_ix + target_col_ix
            step_ct = 0
            step_ix = tail_col_ix + 1

            while step_ct < steps and step_ix <= target_col_ix:
                mark_trail(tail_row_ix, step_ix)

                step_ix = step_ix + 1
                step_ct = step_ct + 1

            tail = [tail_row_ix, target_col_ix]
        elif head_col_ix > tail_col_ix + 1:
            mark_trail(tail_row_ix, tail_col_ix)

            target_row_ix = head_row_ix
            tail = [target_row_ix, tail_col_ix + 1]
            new_tail_row_ix = tail[0]
            new_tail_col_ix = tail[1]

            set_tail(new_tail_row_ix, new_tail_col_ix)

            if head_col_ix > new_tail_col_ix:
                tail = new_tail_destination(head, tail, direction, True)
    # Up
    elif direction == 'U':
        if head_col_ix == tail_col_ix:
            if recurse:
                # This means 't' came from a diagonal and we fed the new values back into this function
                # We need to start from one row down, due to the step_ix increment below
                tail_row_ix = tail_row_ix + 1

            # 'h' and 't' are on the same column
            target_row_ix = head_row_ix + 1
            steps = tail_row_ix - target_row_ix
            step_ct = 0
            step_ix = tail_row_ix - 1

            while step_ct < steps and step_ix >= target_row_ix:
                mark_trail(step_ix, tail_col_ix)

                step_ix = step_ix - 1
                step_ct = step_ct + 1

            tail = [target_row_ix, tail_col_ix]
        elif head_row_ix < tail_row_ix - 1:
            # 'h' has travelled at least two rows up
            # 't' will jump diagonally into h's column
            mark_trail(tail_row_ix, tail_col_ix)
            
            target_row_ix = tail_row_ix - 1
            tail = [target_row_ix, head_col_ix]
            new_tail_row_ix = tail[0]
            new_tail_col_ix = tail[1]
            
            set_tail(new_tail_row_ix, new_tail_col_ix)

            if head_row_ix < new_tail_row_ix:
                # 'h' and 't' are in the same column now, but 't' may have more to travel
                # Feed this new data back into this same function with recurse = True
                tail = new_tail_destination(head, tail, direction, True)
    # Down
    else:
        if head_col_ix == tail_col_ix:
            if recurse:
                tail_row_ix = tail_row_ix - 1
            target_row_ix = head_row_ix - 1
            steps = target_row_ix - tail_row_ix
            step_ct = 0
            step_ix = tail_row_ix + 1

            while step_ct < steps and step_ix <= target_row_ix:
                mark_trail(step_ix, tail_col_ix)

                step_ix = step_ix + 1
                step_ct = step_ct + 1

            tail = [target_row_ix, tail_col_ix]
        elif head_row_ix > tail_row_ix + 1:
            mark_trail(tail_row_ix, tail_col_ix)

            target_row_ix = tail_row_ix + 1
            tail = [target_row_ix, head_col_ix]
            new_tail_row_ix = tail[0]
            new_tail_col_ix = tail[1]

            set_tail(new_tail_row_ix, new_tail_col_ix)

            if head_row_ix > new_tail_row_ix:
                tail = new_tail_destination(head, tail, direction, True)

    new_tail_row_ix = tail[0]
    new_tail_col_ix = tail[1]

    set_tail(new_tail_row_ix, new_tail_col_ix)

    return tail

In [840]:
# Default (current) location index points
head = [0, 0]
tail = [0, 0]

for m in data:
    move = m.split(' ')
    direction = move[0]
    steps = int(move[1])

    head = new_head_destination(head, direction, steps)
    tail = get_tail_position()
    
    if head != tail:
        tail = new_tail_destination(head, tail, direction)

grid_rows = len(grid)
grid_cols = len(grid[0])

print(f"The grid has {grid_rows} rows and {grid_cols} columns")
print(f"Head position: {head}")
print(f"Tail position: {tail}")

The grid has 348 rows and 305 columns
Head position: [313, 20]
Tail position: [312, 19]


In [841]:
df = pd.DataFrame(grid)
pd.set_option('display.max_rows', grid_rows)
pd.set_option('display.max_columns', grid_cols)
pd.set_option('display.width', 1250)
print(df)

    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99  100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 

In [842]:
trail_ct = 1 # Starting at 1 because of the 's' (starting) point

for r in grid:
    for c in r:
        if '#' in c:
            trail_ct = trail_ct + 1

print(f"Answer (part one) - The tail of the rope visited a total of {trail_ct} positions at least once")

Answer (part one) - The tail of the rope visited a total of 6384 positions at least once
